<a href="https://colab.research.google.com/github/hungngo97/Sparse-Regression/blob/master/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# STEP1 : Read .mat files
# https://scipy-cookbook.readthedocs.io/items/Reading_mat_files.html
#Saving .mat file to CSV and upload to github to read data for future usage
# https://towardsdatascience.com/3-ways-to-load-csv-files-into-colab-7c14fcbdcb92



# TODO: Try to visualize Kutz's Matlab code
# [m,n,k]=size(BZ_tensor); % x vs y vs time data
# for j=1:k
# A=BZ_tensor(:,:,j);
# pcolor(A), shading interp, pause(0.2)
# end

# STEP2: Sparse Regression
# TODO: Read Kutz's notes

# STep3: KL divergence
# https://bigdatascientistblog.wordpress.com/2017/09/11/a-simple-introduction-to-kullback-leibler-divergence-through-python-code/

# tep4: AIC BIC
# https://www.reddit.com/r/statistics/comments/5h2rbw/does_python_have_a_package_for_aicbic/
# https://en.wikipedia.org/wiki/Akaike_information_criterion
#   https://en.wikipedia.org/wiki/Bayesian_information_criterion